In [ ]:
import os
os._exit(00)

In [1]:
import os
import pandas as pd
import numpy as np
import ray
import sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
sys.path.insert(0, "/home/erskordi/projects/Autonomous_Fed/")
sys.path.insert(0, "/home/erskordi/projects/Autonomous_Fed/scripts")

import ray.rllib.algorithms.ppo as ppo

from ray import tune, serve, air
from ray.tune.registry import register_env
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.policy.policy import Policy

from config import Config
from env import AutonomousFed
from data_prep import gen_seq, series_to_supervised, plotting, DataPrep
from sim import TF_VAE_Model

tf1, tf, tfv = try_import_tf()
tf1.enable_eager_execution()

RAY_PICKLE_VERBOSE_DEBUG=1
os.environ['PYTHONWARNINGS'] = "ignore::DeprecationWarning"
os.environ['RAY_SERVE_QUEUE_LENGTH_RESPONSE_DEADLINE_S'] = '3'

In [2]:
if ray.is_initialized():
    ray.shutdown()

ray.init(num_cpus=4)

specifications_set = input("Choose specifications set {A, B, C}: ")
# Initialize Ray Serve
serve.start()

# Load the models based on the specifications set
encoder_path = "/home/erskordi/projects/Autonomous_Fed/saved_models/encoder_FedModel_A.keras"
decoder_path = "/home/erskordi/projects/Autonomous_Fed/saved_models/decoder_FedModel_A.keras"
path = [encoder_path, decoder_path]

# Deploy the models
serve.run(target=TF_VAE_Model.bind(path), logging_config={"log_level":"ERROR"})

df, scaler = DataPrep().read_data(specifications_set=specifications_set)

env_config = {'start_date': '2021-07-01', 
              'end_date': '2050-12-31', 
              'model_type': 'VAE',
              'action_specifications': 'ir_omega_equals',
              'omega_pi': 0.5,
              'omega_psi': 0.5,
              'specifications_set': specifications_set,
              'df': df,
              'scaler': scaler,
              'model_config': Config()}

env_name = "AutonomousFed"
register_env(env_name, lambda config: AutonomousFed(env_config))

config = ppo.PPOConfig().\
    environment(env_name, disable_env_checking=True).\
        rollouts(num_rollout_workers=0)
config = config.to_dict()
config["num_workers"] = 0

2024-03-12 11:24:03,735	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Choose specifications set {A, B, C}:  A


(ProxyActor pid=10101) INFO 2024-03-12 11:24:11,334 proxy 172.25.64.98 proxy.py:1143 - Proxy actor a5e2bff20dbe696cd0136e5201000000 starting on node 9055219526e677a21772f41cc9028a2eb5aab7e8b2f750c3cf314c59.
(ProxyActor pid=10101) INFO 2024-03-12 11:24:11,342 proxy 172.25.64.98 proxy.py:1357 - Starting HTTP server on node: 9055219526e677a21772f41cc9028a2eb5aab7e8b2f750c3cf314c59 listening on port 8000
(ProxyActor pid=10101) INFO:     Started server process [10101]
(ServeController pid=10050) INFO 2024-03-12 11:24:11,564 controller 10050 deployment_state.py:1545 - Deploying new version of deployment TF_VAE_Model in application 'default'. Setting initial target number of replicas to 1.
(ServeController pid=10050) INFO 2024-03-12 11:24:11,693 controller 10050 deployment_state.py:1829 - Adding 1 replica to deployment TF_VAE_Model in application 'default'.


In [10]:
# Use the `from_checkpoint` utility of the Policy class:
chckpt_equals = '/home/erskordi/ray_results/PPO_2024-03-11_13-14-00/PPO_AutonomousFed_c1045_00000_0_2024-03-11_13-14-00/checkpoint_000004/policies/default_policy/'
chckpt_not_equals = '/home/erskordi/ray_results/PPO_2024-03-12_11-27-57/PPO_AutonomousFed_1a543_00000_0_2024-03-12_11-27-57/checkpoint_000004/policies/default_policy/'
chckpt_psi_action = ''
chckpt_all_actions = ''
my_restored_policy = Policy.from_checkpoint(chckpt_not_equals)

In [11]:
from env import AutonomousFed

env = AutonomousFed(env_config)

In [12]:
epsilon = 1e-10

obs, info = env.reset()
obs

array([5.33, 9.16], dtype=float16)

In [13]:
results_all_equals = {'inflation':[],
                      'GDP_gap':[],
                      'interest_rate':[]}
terminated = False
while not terminated:
    action = my_restored_policy.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action[0])
    results_all_equals['inflation'].append(obs[0])
    results_all_equals['GDP_gap'].append(obs[1])
    results_all_equals['interest_rate'].append(action[0][0])
    print(obs, action[0][0])

[1.296 4.85 ] 0.16113907
[1.045 1.989] 0.30468544
[0.2043 3.154 ] -0.2852373
[0.089 4.21 ] -0.67046094
[0.05643 4.73   ] 0.62979496
[0.08887 4.637  ] -0.47983772
[0.0654 4.52  ] -0.9317903
[0.0836 4.484 ] -0.20787735
[0.0865 4.387 ] -0.24255864
[0.09705 4.35   ] -0.5014633
[0.06726 4.74   ] -0.31990996
[0.07837 4.535  ] -0.14632653
[0.1017 4.23  ] -0.49021953
[0.04874 5.004  ] -0.60698867
[0.06445 4.74   ] -0.6782799
[0.0713 4.594 ] -0.9838592
[0.04852 4.83   ] -1.0730362
[0.09595 4.61   ] -0.31312013
[0.0747 4.527 ] 0.10228887
[0.03998 4.633  ] -1.4538124
[0.08746 4.383  ] -0.31114513
[0.09064 4.676  ] -0.09115371
[0.0874 4.504 ] -0.16475324
[0.08417 4.703  ] -1.0500436
[0.07465 4.805  ] -0.3538544
[0.0503 4.785 ] -1.1350507
[0.07983 4.45   ] -0.56765145
[0.1115 4.418 ] -0.26849565
[0.08685 4.54   ] -0.61247206
[0.10333 4.477  ] -0.1357108
[0.05933 4.72   ] -0.7312299
[0.0949 4.44  ] 0.054606944
[0.07587 4.516  ] -0.8507584
[0.0658 4.69  ] 0.5467051
[0.04135 4.934  ] -1.0442648
[0.093

(ServeController pid=10050) INFO 2024-03-12 18:45:00,017 controller 10050 deployment_state.py:1545 - Deploying new version of deployment TF_VAE_Model in application 'default'. Setting initial target number of replicas to 1.
(ServeController pid=10050) INFO 2024-03-12 18:45:00,497 controller 10050 deployment_state.py:1708 - Stopping 1 replicas of deployment 'TF_VAE_Model' in application 'default' with outdated versions.
(ServeController pid=10050) INFO 2024-03-12 18:45:03,263 controller 10050 deployment_state.py:2187 - Replica default#TF_VAE_Model#kRjBWm is stopped.
(ServeController pid=10050) INFO 2024-03-12 18:45:03,263 controller 10050 deployment_state.py:1829 - Adding 1 replica to deployment TF_VAE_Model in application 'default'.
(ServeController pid=10050) INFO 2024-03-12 18:48:44,745 controller 10050 deployment_state.py:1545 - Deploying new version of deployment TF_VAE_Model in application 'default'. Setting initial target number of replicas to 1.
(ServeController pid=10050) INFO 

In [7]:
df = pd.DataFrame(results_all_equals)
df.to_csv('/home/erskordi/projects/Autonomous_Fed/scripts/results/omegas_equal.csv')